# C3.2. Тонкости обработки исключений. Собственные классы исключений.

Классы +-- SystemExit +-- KeyboardInterrupt +-- GeneratorExit — являются исключениями, которые нельзя поймать, т.к. их возникновение не зависит от выполнения программы. А все, что наследуются от Exception, можно отловить и обработать

In [1]:
try:
    raise ZeroDivisionError  # возбуждаем исключение ZeroDivisionError
except ArithmeticError:  # ловим его родителя
    print("Hello from arithmetic error")

Hello from arithmetic error


 если вы хотите ловить сначала сами классы, а потом любой другой из наследуемых, то убедитесь, что вы идёте от конкретного к абстрактному.

Вот правильный пример, для наглядности:

In [2]:
try:
    raise ZeroDivisionError
except ZeroDivisionError:  # сначала пытаемся поймать наследника
    print("Zero division error")
except ArithmeticError:  # потом ловим потомка
    print("Arithmetic error")

Zero division error


In [3]:
# Принцип написания и отлова собственного исключения следующий:

class MyException(Exception):  # создаём пустой класс – исключения
    pass
try:
    raise MyException("message")  # поднимаем наше исключение
except MyException as e:  # ловим его за хвост как шкодливого котёнка
    print(e)  # выводим информацию об исключении


message


In [ ]:
# собственные исключения с наследованием:

# создаём пустой класс – исключения потомка, наследуемся от exception
class ParentException(Exception):
    pass

# создаём пустой класс – исключение наследника, наследуемся от ParentException
class ChildException(ParentException):
    pass

try:
    raise ChildException("message")  # поднимаем исключение-наследник
except ParentException as e:  # ловим его родителя
    print(e)  # выводим информацию об исключении


класс с самописным исключением не обязательно должен быть пустым. Если вы хотите добавить собственные аргументы в конструктор, дополнительно произвести какие-либо операции, то можете спокойно это делать, главное — не забыть о нескольких нюансах:

In [ ]:
class ParentException(Exception):
    # допишем к нашему пустому классу конструктор, который будет печатать дополнительно в консоль информацию об ошибке.
    def __init__(self, message, error):
        super().__init__(message)  # помним про вызов конструктора родительского класса
        print(f"Errors: {error}")  # печатаем ошибку


# создаём пустой класс – исключение наследника, наследуемся от ParentException
class ChildException(ParentException):
    def __init__(self, message, error):
        super().__init__(message, error)


try:
    # поднимаем исключение-наследник, передаём дополнительный аргумент
    raise ChildException("message", "error")
except ParentException as e:
    print(e)  # выводим информацию об исключении
# Сначала мы увидим то, что напишет нам конструктор родительского класса, а потом уже увидим сообщение об ошибке.


### ***Давайте подведём итоги:***

- Исключения — это такие особенные классы, 
    которые как и любые классы можно наследовать. 
    Если вы хотите ловить несколько исключений, 
    то сначала ловите потомков, а потом родителей, 
    чтобы ничего не упустить.
- Чтобы создать собственный класс, нужно просто написать пустой класс 
    и наследовать его от класса Exception, этого будет достаточно.
- Не обязательно отлавливать сам класс, при необходимости можно 
    отлавливать его родителя, это тоже будет работать, 
    но вы можете упустить важную информацию.


Задание 3.2.5

Задание на самопроверку.

Создать класс Square. Добавить в конструктор класса Square собственное исключение NonPositiveDigitException, унаследованное от ValueError, которое будет срабатывать каждый раз, когда сторона квадрата меньше или равна 0.


In [6]:
class NonPositiveDigitException(ValueError):
    def __str__(self):
        return "сторона квадрата меньше или равна 0"

class Square:
    def __init__(self, side):
        self.side = side
    def get_area(self):
        try:
            if self.side <= 0:
                raise NonPositiveDigitException
            else: 
                return self.side**2
        except ValueError as e:
            print(e)

s = Square(0)
print("s area = " + str(s.get_area()))

сторона квадрата меньше или равна 0
s area = None


In [10]:
# Эталон решения

class NonPositiveDigitException(ValueError):
    pass

class Square:
    def __init__(self, a):
        if a <= 0:
            raise NonPositiveDigitException(
                'Неправильно указанна сторона квадрата')

s = Square(1)

# C3.3. Работа с импортом

**Как импортировать модуль**

    import sys
    import os

Импорт происходит с помощью зарезервированного слова **import** название **модуля**.

Для того, чтобы обратиться к содержимому модуля, нужно после его имени поставить точку и указать нужный вам элемент.

    import os
    
    print(os.getcwd())  # получить текущую директорию
    print(os.listdir())  # получить список файлов текущей директории

*Не следует импортировать модули в одну строку, каждый отдельный модуль должен импортировать на отдельной строке:*

    # правильно
    import os
    import sys
    
    # неправильно
    import os, sys

Но если вы хотите импортировать, например, несколько функций из одного модуля, можно перечислить их через запятую:

    from subprocess import Popen, PIPE

Также для импорта существует несколько различных способов записи, но делают они тоже самое: импорт всего модуля и обращение к функции через имя модуля:

    import math
    print(math.pi)  # 3.141592653589793

Импортируем всё из модуля, это позволяет сократить запись, и обращаться к функциям, не указывая имя модуля:

    from math import *  # импортируем всё из модуля math
    print(pi)  # 3.141592653589793

Даем новое имя модулю, это позволяет избежать ошибок, если у вас есть функции, которая называется так же как модуль, то вы можете дать новое имя модулю для данного скрипта:

    import math as m  # использование нового имени для обращения к импортированному модулю
    print(m.pi)  # 3.141592653589793

Даем новое имя функции или переменной, это позволяет избежать ошибок, если у вас есть одноименная функция в вашей программе, и вы не хотите её переименовать:

    from math import pi as PI
    print(PI)  # 3.141592653589793





## Как правильно составить модуль?

1. Вся основная логика модуля заключена в отдельные функции или классы. На глобальном уровне могут быть объявлены только константы или необходимые для инициализации модуля операции.

2. Если вы планируете, что модуль могут запускать как самостоятельный скрипт – используйте следующую инструкцию: 

            if __name__ == '__main__':.

    *Как это работает?* Ваш скрипт может выполняться и самостоятельно, а может быть импортирован как модуль другим скриптом. Чтобы выделить код, который не должен выполняться при импорте его следует поместить в условный оператор с условием 
  
            if __name__ == '__main__':.
        
3. Хорошая структура модуля выглядит следующим образом:                                                                                                                         

- Docstring (описание) модуля.
- Область импорта:
  - импорты системных библиотек;
  - импорты стандартных пакетов (из PyPI);
  - импорты ваших модулей (локальных).
- Область объявление глобальных констант.
- Инициализация модуля.
- Область определения функций и классов.
- Функции.

                if __name__ == '__main__' 
                
  (метод main) по желанию (это одни из немногих нюансов при работе с собственными модулями).

***ВАЖНО***: *Чтобы модули заработали правильно, их нужно хранить в той же папке, в которой вы запускаете главный скрипт, иначе Python не найдёт ваш модуль!*


In [14]:
import math
import time
f = math.trunc(math.fmod(math.fabs(-10000000), 55)+0.3)
f
t = time.asctime()
t

'Sun Jul 25 02:17:50 2021'

In [18]:
import time

i = 10
while i != 0:
  print(i)
  i -= 1
  time.sleep(1)
print("Time's up!")



10
9
8
7
6
5
4
3
2
1
Time's up!


## **Задание 3.3.7**

*Задание на самопроверку.*

Вам нужно написать два модуля:

+ Первый должен содержать число Пи в виде константы 3.14 и две функции, которые будут считать площадь круга и прямоугольника.
+ Второй модуль должен импортировать первый, далее запрашивать у пользователя размеры круга и квадрата. В результате выводить, какая из фигур больше.


In [ ]:
PI = 3.14


def circle_area(r):
   return PI * (r ** 2)


def rect_area(a, b):
   return a * b


if __name__ == '__main__':
   # проверяем работоспособность функции, дальнейшая часть не будет импортирована
   # если ответы будут отличаться, то будет вызвана ошибка
   assert circle_area(5) == 78.5
   assert rect_area(5, 4) == 20

##############################



In [ ]:
from module_name import *


def main():
   r = input('Введите радиус круга:\n')

   a = input('Введите длину прямоугольника:\n')
   b = input('Введите ширину прямоугольника:\n')

   if circle_area(r) > rect_area(a, b):
       print('Площадь круга больше')
   else:
       print('Площадь прямоугольника больше')


# C3.4. Работа с файлами

Чтобы поработать с путями есть модуль os. Функция os.chdir() позволяет нам изменить директорию, которую мы в данный момент используем. Если вам нужно знать, какой путь вы в данный момент используете, для этого нужно вызвать os.getcwd().

In [4]:
import os
# получить текущий путь
start_path = os.getcwd()
print(start_path)  # /home/nbuser/library
# Далее попробуем подняться на директорию выше:
os.chdir("..")  # подняться на один уровень выше
os.getcwd()  # '/home/nbuser'
# Теперь вернемся в ту директорию, из которой стартовали. Изначально мы сохраняли её в переменной start_path.
os.chdir(start_path)
os.getcwd()  # '/home/nbuser/library'

# С помощью функции os.listdir() можно получить весь список файлов, находящихся в директории. Если не указать никаких аргументов, то будет взята текущая директория.

# список файлов и директорий в папке
# import os
print(os.listdir())
# ['SnapchatLoader', 'FBLoader', 'tmp.py', '.gitignore', 'venv', '.git']

if 'tmp.py' not in os.listdir():
   print("Файл отсутствует в данной директории")


/home/az/DEV/Python/SF_PFS
['.git', 'venv', 'README.md', '.vscode', 'OOP', '.gitignore']
Файл отсутствует в данной директории


**Для того, чтобы склеивать пути с учётом особенностей ОС, следует использовать функцию os.path.join().**

In [5]:
# соединяет пути с учётом особенностей операционной системы
print(start_path)
print(os.path.join(start_path, 'test'))

# /home/nbuser/library
# /home/nbuser/library/test


/home/az/DEV/Python/SF_PFS
/home/az/DEV/Python/SF_PFS/test


## Задание 3.4.3

***Задание на самопроверку.***

Сделайте функцию, которая принимает от пользователя путь и выводит всю информацию о содержимом этой папки. Для реализации используйте функцию встроенного модуля **os.walk()**. Если путь не указан, то сравнение начинается с текущей директории.

In [7]:
# список файлов и директорий в папке
import os
path = input('Enter path to check: ')
print(os.walk(path))


<generator object walk at 0x7f3ad6060f90>


In [13]:
import os


def walk_desc(path=None):
   start_path = path if path is not None else os.getcwd()

   for root, dirs, files in os.walk(start_path):
       print("Текущая директория", root)
       print("---")

       if dirs:
           print("Список папок", dirs)
       else:
           print("Папок нет")
       print("---")

       if files:
           print("Список файлов", files)
       else:
           print("Файлов нет")
       print("---")

       if files and dirs:
           print("Все пути:")
       for f in files:
           print("Файл ", os.path.join(root, f))
       for d in dirs:
           print("Папка ", os.path.join(root, d))
       print("===")

path = './OOP/C1_Practice'
walk_desc(path)


Текущая директория ./OOP/C1_Practice
---
Список папок ['__pycache__']
---
Список файлов ['testRectangle.py', 'cat.py', 'rectangle.py', 'cats.py']
---
Все пути:
Файл  ./OOP/C1_Practice/testRectangle.py
Файл  ./OOP/C1_Practice/cat.py
Файл  ./OOP/C1_Practice/rectangle.py
Файл  ./OOP/C1_Practice/cats.py
Папка  ./OOP/C1_Practice/__pycache__
===
Текущая директория ./OOP/C1_Practice/__pycache__
---
Папок нет
---
Список файлов ['cat.cpython-38.pyc', 'rectangle.cpython-38.pyc']
---
Файл  ./OOP/C1_Practice/__pycache__/cat.cpython-38.pyc
Файл  ./OOP/C1_Practice/__pycache__/rectangle.cpython-38.pyc
===


## Работа с файлами

*Python* «из коробки» располагает достаточно широким набором инструментов для работы с файлами. Для того чтобы начать работать с файлом, надо его открыть с помощью команды специальной функции **open**.

    f = open('path/to/file', 'filemode', encoding='utf8')



Давайте по порядку разберем все аргументы:

1. path/to/file — путь к файлу может быть относительным или абсолютным. Можно указывать в Unix-стиле (path/to/file) или в Windows-стиле (path\to\file).
2. filemode — режим, в котором файл нужно открывать.
    Записывается в виде строки, состоит из следующих букв:
    - r — открыть на чтение (по умолчанию);
    - w — перезаписать и открыть на запись (если файла нет, то он создастся);
    - x — создать и открыть на запись (если уже есть — исключение);
    - a — открыть на дозапись (указатель будет поставлен в конец);
    - t — открыть в текстовом виде (по умолчанию);
    - b — открыть в бинарном виде.
3. encoding — указание, в какой кодировке файл записан (utf8, cp1251 и т. д.) По умолчанию стоит utf-8.


In [14]:
f = open('test.txt', 'w', encoding='utf8')

# Запишем в файл строку
f.write("This is a test string\n")
f.write("This is a new string\n")

21

После вызова команды write ваши данные не сразу попадут и сохранятся в файл. Связанно это с особенностями внутренней работы операционных систем. Если для вас критично своевременно попадание информации на жесткий диск компьютера, то после записи вызывайте f.flush() или закрывайте файл. Закрыть файл, можно с помощью метода close().

In [17]:
# обязательно нужно закрыть файл иначе он будет заблокирован ОС
f.close()

f = open('test.txt', 'r', encoding='utf8')
print(f.read(10))  # This is a
# считали остаток файла
print(f.read())  # test string\nThis is a new string\n
# обязательно закрываем файл
f.close()


This is a 
test string
This is a new string



Зачастую с файлами удобнее работать построчно, поэтому для этого есть отдельные методы:

  - writelines — записывает список строк в файл;
  - readline — считывает из файла одну строку и возвращает её;
  - readlines — считывает из файла все строки в список и возвращает их.

Метод **f.writelines(sequence)** не будет сам за вас дописывать символ конца строки **(‘\n’)**. Поэтому при необходимости его нужно прописать вручную.

In [18]:
f = open('test.txt', 'a', encoding='utf8')  # открываем файл на дозапись

sequence = ["other string\n", "123\n", "test test\n"]
# берет строки из sequence и записывает в файл (без переносов)
f.writelines(sequence)

f.close()


In [19]:
f = open('test.txt', 'r', encoding='utf8')

print(f.readlines())  # считывает все строки в список и возвращает список

f.close()


['This is a test string\n', 'This is a new string\n', 'other string\n', '123\n', 'test test\n']


In [20]:
f = open('test.txt', 'r', encoding='utf8')

print(f.readline())  # This is a test string
print(f.read(4))  # This
print(f.readline())  # is a new string

f.close()


This is a test string

This
 is a new string



## Файл как итератор

***Объект файл является итератором, поэтому его можно использовать в цикле for.***

In [21]:
f = open('test.txt')  # можно перечислять строки в файле
for line in f:
    print(line, end='')

# This is a test string
# This is a new string
# other string
# 123
# test test

f.close()


This is a test string
This is a new string
other string
123
test test


## Менеджер контекста with

Для явного указания места работы с файлом, а также чтобы не забывать закрывать файл после обработки, существует менеджер контекста **with**.

In [24]:
# В блоке менеджера контекста открытый файл «жив» и с ним можно работать, при выходе из блока - файл закрывается.
with open("test.txt", 'rb') as f:
    a = f.read(10)
    b = f.read(23)
print(a, b)
# f.read(3)  # Error!

b'This is a ' b'test string\nThis is a n'


## Задание 3.4.4

*Задание на самопроверку.*

Создайте любой файл на операционной системе под название input.txt и построчно перепишите его в файл output.txt.

In [26]:
with open('input.txt', 'r') as input_file:
   with open('output.txt', 'w') as output_file:
       for line in input_file:
           output_file.write(line)


## Задание 3.4.5

*Задание на самопроверку.*

Дан файл **numbers.txt**, компоненты которого являются действительными числами (файл создайте самостоятельно и заполните любыми числам, в одной строке одно число). Найдите сумму наибольшего и наименьшего из значений и запишите результат в файл **output.txt**.

In [28]:
filename = 'numbers.txt'
output = 'output.txt'

with open(filename) as f:
   min_ = max_ = float(f.readline())  # считали первое число
   for line in f:
       num = float(line)
       if num > max_:
           max_ = num
       elif num < min_:
           min_ = num

   sum_ = min_ + max_

with open(output, 'w') as f:
   f.write(str(sum_))
   f.write('\n')


## Задание 3.4.6

*Задание на самопроверку.*

В текстовый файл построчно записаны фамилии и имена учащихся класса и их оценки за контрольную. Выведите на экран всех учащихся, чья оценка меньше 3 баллов. Cодержание файла:


Иванов О. 4
Петров И. 3
Дмитриев Н. 2
Смирнова О. 4
Керченских В. 5
Котов Д. 2
Бирюкова Н. 1
Данилов П. 3
Аранских В. 5
Лемонов Ю. 2
Олегова К. 4

In [42]:
count = 0
names = []
st_names = ''
for line in open("input.txt"):
   points = int(line.split()[-1])
   if points < 3:
       count += 1
       names.append(line[:-3])
print('Троечником в классе: ' + str(count) + " чел.")
for i in names:
  st_names += i + ' '
print("Это " + st_names)

Троечником в классе: 4 чел.
Это Дмитриев Н. Котов Д. Бирюкова Н. Лемонов Ю. 


## Задание 3.4.7

*Задание на самопроверку.*

Выполните реверсирование строк файла (перестановка строк файла в обратном порядке).

In [43]:
with open('input.txt', 'r') as input_file:
   with open('output.txt', 'w') as output_file:
       for line in reversed(input_file.readlines()):
           output_file.write(line)


# C3.5. Контекстные менеджеры. Ключевое слово with, принципы создания собственных контекстных менеджеров.

## **Контекстный менеджер** — определенная структура в языке Python (класс или генератор), основывающаяся на главном принципе: при его открытии и закрытии срабатывает заранее написанный программистом код.

Открытие происходит при входе в блок с помощью ключевого слова with. Закрытие происходит, когда блок заканчивается. Например, на входе — открывается файл, на выходе — закрывается.

In [ ]:
with open("file.bin", "wt") as f:  # открываем файл с помощью with
    f.write("abcdefg")

Чтобы написать контекстный менеджер нужно всего лишь помнить о нескольких вещах:

  1. Нужно создать класс и написать в нём метод __enter__. Код в этом методе будет выполняться при входе в контекстный менеджер (при создании объекта с ключевым словом with).
  2. Написать метод __exit__. Этот метод будет выполнять код, помещённый в него, на выходе.
  3. Добавить в этот метод три дополнительных аргумента помимо self — exc_type, exc_val, exc_tb. Зачем они нужны, расскажу чуть позже.


In [44]:
from datetime import datetime
import time  # проверять действие измерителя будем с помощью библиотеки time


# вся суть этого измерителя заключается в том, что мы считаем разницу в секундах между открытием и закрытием контекстного менеджера
class Timer:
    def __init__(self):
        pass

    def __enter__(self):  # этот метод вызывается при запуске с помощью with. Если вы хотите вернуть какой-то объект, чтобы потом работать с ним в контекстном менеджере, как в примере с файлом, то просто верните этот объект через return
        self.start = datetime.utcnow()
        return None

    # этот метод срабатывает при выходе из контекстного менеджера
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(
            f"Time passed: {(datetime.utcnow() - self.start).total_seconds()}")


with Timer():
    time.sleep(2)  # засыпаем на 2 секунды


Time passed: 2.000557


В дополнение к комментариям в коде ещё хотелось бы сказать пару слов об аргументах метода 

    __exit__.

О каждом из них по порядку:

- ***exc_type*** — это тип исключения, из-за которого вылетел контекстный менеджер. Если всё прошло успешно, то значение этого аргумента будет None.
- ***exc_val*** — сообщение в исключении. Аналогично: если всё прошло успешно, этот аргумент будет **None**.
- ***exc_tb*** — объект сообщения от интерпретатора. Лучшего его вообще не трогать, если вы не разработчик языка, но тем не менее он всегда ждёт вас здесь. Возможно когда-то, после нашего курса вы…


### **Возможность создания контекстных менеджеров через генераторы**

Разница только в том, что контекстный менеджер на генераторах — это функция (для особо привередливых — генератор). В ней до 

    yield 
выполняется код, который мы могли бы записать в 

    __enter__ 
, если бы делали контекстный менеджер в виде класса, а после 

    yield 
пишем код, который выполнился бы в 

    __exit__. 
То есть до **yield** — всё что произойдёт при входе, после — всё, что на выходе. Вот и вся разница.

In [45]:
from datetime import datetime
import time


from contextlib import contextmanager  # импортируем нужный нам декоратор


@contextmanager  # оборачиваем функцию в декоратор contextmanager
def timer():
    start = datetime.utcnow()
    # если вам нужно что-то вернуть через контекстный менеджер, просто вставьте этот объект сюда.
    yield
    print(f"Time passed: {(datetime.utcnow() - start).total_seconds()}")


with timer():
    time.sleep(2)


Time passed: 2.002084


## Задание 3.5.6

**Напишите контекстный менеджер, который умеет безопасно работать с файлами.**

При входе в контекстный менеджер передаются два аргумента: первый — путь к файлу который надо открыть, второй — тип открываемого файла (для записи, для чтения и т. д.). При выходе из контекстного менеджера файл должен закрываться. (Эталоном работы можно считать контекстный менеджер open).

In [47]:
class OpenFile:
    def __init__(self, path, type):
        self.file = open(path, type)

    def __enter__(self):
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.file.close()


with OpenFile('hello.txt', 'wt') as f:
    f.write('Мой контекстный менеджер делает тоже самое!')
